In [1]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import numpy as np
from sentence_transformers import SentenceTransformer

df = pd.read_csv("cleaned_FE_data.csv")
df.info()

for col in ['description_clean', 'tags_clean', 'text']:
    df[col] = df[col].fillna("")

text_model = SentenceTransformer('all-MiniLM-L6-v2')
X_text = text_model.encode(df['text'].tolist(), show_progress_bar=True)
X_desc = text_model.encode(df['description_clean'].tolist(), show_progress_bar=True)
X_tags = text_model.encode(df['tags_clean'].tolist(), show_progress_bar=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2531 entries, 0 to 2530
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   video_id            2531 non-null   object 
 1   title               2531 non-null   object 
 2   description         2392 non-null   object 
 3   published_at        2531 non-null   object 
 4   channel_id          2531 non-null   object 
 5   channel_title       2531 non-null   object 
 6   tags                2531 non-null   object 
 7   view_count          2531 non-null   float64
 8   like_count          2531 non-null   float64
 9   comment_count       2531 non-null   float64
 10  duration            2531 non-null   object 
 11  category_id         2531 non-null   int64  
 12  live_broadcast      2531 non-null   object 
 13  subscriber_count    2531 non-null   int64  
 14  video_count         2531 non-null   int64  
 15  view_count_total    2531 non-null   int64  
 16  has_de

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
structured_features = [
    'title_sentiment', 'is_clickbait', 'has_number', 'has_punctuation',
    'publish_hour', 'publish_weekday', 'is_weekday', 'publish_month', 'is_holiday',
    'cluster', 'tag_count', 'duration_seconds'
]

X_struct = df[structured_features].fillna(0).values

X_all = np.hstack([X_desc, X_tags, X_text, X_struct])
y = df['like_view_ratio'].astype(float)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)

# Define models
models = {
    "RandomForest": RandomForestRegressor(
        n_estimators=150,
        max_depth=20,
        min_samples_leaf=10,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ),
    "XGBoost": XGBRegressor(
        n_estimators=100,
        learning_rate=0.05,
        max_depth=15,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),
   "LinearRegression": LinearRegression()
}
# Train and evaluate
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    for split, X_split, y_split in [("Train", X_train, y_train), ("Test", X_test, y_test)]:
        y_pred = model.predict(X_split)
        mae = mean_absolute_error(y_split, y_pred)
        rmse = np.sqrt(mean_squared_error(y_split, y_pred))
        r2 = r2_score(y_split, y_pred)
        results.append({
            "Model": name,
            "Dataset": split,
            "MAE": mae,
            "RMSE": rmse,
            "R2": r2
        })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

           Model Dataset      MAE     RMSE        R2
    RandomForest   Train 0.096885 0.131235  0.590460
    RandomForest    Test 0.137452 0.181671  0.203187
         XGBoost   Train 0.006033 0.009400  0.997899
         XGBoost    Test 0.128210 0.171412  0.290638
LinearRegression   Train 0.080702 0.103447  0.745535
LinearRegression    Test 0.191332 0.244074 -0.438240
